# Text Generation

In [1]:
# ! pip install evaluate
# !pip install transformers datasets tokenizers seqeval -q

In [2]:
# Import libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
from datasets import load_dataset
import evaluate

import transformers
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline

from sklearn.model_selection import train_test_split
import os

In [3]:
df_nlp = pd.read_json('../data/banking_responses.jsonl', lines = True)

In [4]:
df_nlp.head()

,key,transcript,convenient,speed,informative
0,1,"I had to wait a while, but the representative ...",Representative clearly explained,Wait a while,Completed the account setup quickly.
1,2,"The app was easy to use, and I transferred fun...",App was easy to use,Transferred funds instantly,Clarified the mortgage plan benefits.
2,3,The representative took some time but explaine...,Explained the mortgage benefits thoroughly,Representative took some time,Set up my account seamlessly.
3,4,Customer service quickly resolved my issue wit...,Resolved my issue,Quickly resolved,Set up my account seamlessly.
4,5,"The wait was short, and the staff quickly set ...",Staff quickly set up my account,Wait was short,Completed the account setup quickly.


# Transform Data

In [5]:
df_nlp['tokens'] = df_nlp['transcript'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['convenient'] = df_nlp['convenient'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['speed'] = df_nlp['speed'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['informative'] = df_nlp['informative'].apply(lambda x: re.findall(r'[.?]|\w+', x))

In [6]:
def map_tags(lst, input:str):
  dct = {}
  print(lst)
  begin = ""
  inbet = ""
  if input == "convenient":
    begin = 1
    inbet = 2
  elif input == "speed":
    begin = 3
    inbet = 4
  elif input == "informative":
    begin = 5
    inbet = 6
  dct[lst[0]] = begin
  for i in range(1, len(lst)):
    dct[lst[i]] = inbet
  # print(dct)
  return dct

In [7]:
df_nlp['convenient'] = df_nlp['convenient'].apply(lambda x: map_tags(x, 'convenient'))
df_nlp['speed'] = df_nlp['speed'].apply(lambda x: map_tags(x, 'speed'))
df_nlp['informative'] = df_nlp['informative'].apply(lambda x: map_tags(x, 'informative'))

['Representative', 'clearly', 'explained']
['App', 'was', 'easy', 'to', 'use']
['Explained', 'the', 'mortgage', 'benefits', 'thoroughly']
['Resolved', 'my', 'issue']
['Staff', 'quickly', 'set', 'up', 'my', 'account']
['Representative', 'clearly', 'explained']
['App', 'was', 'easy', 'to', 'use']
['Explained', 'the', 'mortgage', 'benefits', 'thoroughly']
['Resolved', 'my', 'issue']
['Staff', 'quickly', 'set', 'up', 'my', 'account']
['Representative', 'clearly', 'explained']
['App', 'was', 'easy', 'to', 'use']
['Explained', 'the', 'mortgage', 'benefits', 'thoroughly']
['Resolved', 'my', 'issue']
['Staff', 'quickly', 'set', 'up', 'my', 'account']
['Representative', 'clearly', 'explained']
['App', 'was', 'easy', 'to', 'use']
['Explained', 'the', 'mortgage', 'benefits', 'thoroughly']
['Resolved', 'my', 'issue']
['Staff', 'quickly', 'set', 'up', 'my', 'account']
['Representative', 'clearly', 'explained']
['App', 'was', 'easy', 'to', 'use']
['Explained', 'the', 'mortgage', 'benefits', 'thoroug

In [8]:
df_nlp.head()

,key,transcript,convenient,speed,informative,tokens
0,1,"I had to wait a while, but the representative ...","{'Representative': 1, 'clearly': 2, 'explained...","{'Wait': 3, 'a': 4, 'while': 4}","{'Completed': 5, 'the': 6, 'account': 6, 'setu...","[I, had, to, wait, a, while, but, the, represe..."
1,2,"The app was easy to use, and I transferred fun...","{'App': 1, 'was': 2, 'easy': 2, 'to': 2, 'use'...","{'Transferred': 3, 'funds': 4, 'instantly': 4}","{'Clarified': 5, 'the': 6, 'mortgage': 6, 'pla...","[The, app, was, easy, to, use, and, I, transfe..."
2,3,The representative took some time but explaine...,"{'Explained': 1, 'the': 2, 'mortgage': 2, 'ben...","{'Representative': 3, 'took': 4, 'some': 4, 't...","{'Set': 5, 'up': 6, 'my': 6, 'account': 6, 'se...","[The, representative, took, some, time, but, e..."
3,4,Customer service quickly resolved my issue wit...,"{'Resolved': 1, 'my': 2, 'issue': 2}","{'Quickly': 3, 'resolved': 4}","{'Set': 5, 'up': 6, 'my': 6, 'account': 6, 'se...","[Customer, service, quickly, resolved, my, iss..."
4,5,"The wait was short, and the staff quickly set ...","{'Staff': 1, 'quickly': 2, 'set': 2, 'up': 2, ...","{'Wait': 3, 'was': 4, 'short': 4}","{'Completed': 5, 'the': 6, 'account': 6, 'setu...","[The, wait, was, short, and, the, staff, quick..."


In [9]:
df = df_nlp[['key', 'tokens', 'convenient', 'speed', 'informative']]
df = df.rename(columns={'key':'id'})

In [10]:
def map_num(row):
	curr_dict = row['convenient'].copy()
	curr_dict.update(row['speed'])
	curr_dict.update(row['informative'])
	return [0 if curr_dict.get(i) is None else curr_dict.get(i) for i in row['tokens']]


In [11]:
df['ner_tags'] = df.apply(map_num, axis=1)
df.head()

,id,tokens,convenient,speed,informative,ner_tags
0,1,"[I, had, to, wait, a, while, but, the, represe...","{'Representative': 1, 'clearly': 2, 'explained...","{'Wait': 3, 'a': 4, 'while': 4}","{'Completed': 5, 'the': 6, 'account': 6, 'setu...","[0, 0, 0, 0, 4, 4, 0, 6, 0, 2, 2, 6, 0, 0, 0, 6]"
1,2,"[The, app, was, easy, to, use, and, I, transfe...","{'App': 1, 'was': 2, 'easy': 2, 'to': 2, 'use'...","{'Transferred': 3, 'funds': 4, 'instantly': 4}","{'Clarified': 5, 'the': 6, 'mortgage': 6, 'pla...","[0, 0, 2, 2, 2, 2, 0, 0, 0, 4, 4, 6, 0, 0, 2, ..."
2,3,"[The, representative, took, some, time, but, e...","{'Explained': 1, 'the': 2, 'mortgage': 2, 'ben...","{'Representative': 3, 'took': 4, 'some': 4, 't...","{'Set': 5, 'up': 6, 'my': 6, 'account': 6, 'se...","[0, 0, 4, 4, 4, 0, 0, 2, 2, 2, 2, 6]"
3,4,"[Customer, service, quickly, resolved, my, iss...","{'Resolved': 1, 'my': 2, 'issue': 2}","{'Quickly': 3, 'resolved': 4}","{'Set': 5, 'up': 6, 'my': 6, 'account': 6, 'se...","[0, 0, 0, 4, 6, 2, 0, 0, 0, 6]"
4,5,"[The, wait, was, short, and, the, staff, quick...","{'Staff': 1, 'quickly': 2, 'set': 2, 'up': 2, ...","{'Wait': 3, 'was': 4, 'short': 4}","{'Completed': 5, 'the': 6, 'account': 6, 'setu...","[0, 0, 4, 4, 0, 6, 0, 6, 2, 2, 2, 6, 6, 0, 4, ..."


In [12]:
label_list = ['O', 'B-CON', "I-CON", "B-SPD", "I-SPD", "B-INF", "I-INF"]
random_seed = 1
transformers.set_seed(seed=random_seed)

In [13]:
train, test = train_test_split(df, test_size=0.2, random_state=random_seed)
train, val = train_test_split(train, test_size=0.125, random_state=random_seed) # 0.125 x 0.8 = 0.1
train.shape, val.shape, test.shape

((70, 6), (10, 6), (20, 6))

In [14]:
train.to_json('../data/nlp_train.jsonl', orient='records', lines=True)
test.to_json('../data/nlp_test.jsonl', orient='records', lines=True)
val.to_json('../data/nlp_val.jsonl', orient='records', lines=True)

# Load in the dataset

In [15]:
_nlp_data = {
	'train' : '../data/nlp_train.jsonl',
	'validation' : '../data/nlp_val.jsonl',
	'test' : '../data/nlp_test.jsonl'
}

nlp_data =  load_dataset('json', data_files=_nlp_data)
nlp_data

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'convenient', 'speed', 'informative', 'ner_tags'],
        num_rows: 70
    })
    validation: Dataset({
        features: ['id', 'tokens', 'convenient', 'speed', 'informative', 'ner_tags'],
        num_rows: 10
    })
    test: Dataset({
        features: ['id', 'tokens', 'convenient', 'speed', 'informative', 'ner_tags'],
        num_rows: 20
    })
})

# Tokenization

In [16]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

c:\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [18]:
tokenized_datasets = nlp_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

# Model Training

In [19]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [20]:
import numpy as np
import evaluate

# Assuming label_list is defined elsewhere in your code
metric = evaluate.load("seqeval")  # Load the seqeval metric

def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    # Get the predicted class labels by taking argmax
    pred_logits = np.argmax(pred_logits, axis=2)

    # Remove all values where the label is -100 and create predictions
    predictions = [
        [label_list[eval_pred] for (eval_pred, l) in zip(prediction, label) if l != -100 and 0 <= eval_pred < len(label_list)]
        for prediction, label in zip(pred_logits, labels)
    ]

    # Remove all values where the label is -100 and create true labels
    true_labels = [
        [label_list[l] for (l, eval_pred) in zip(label, prediction) if l != -100 and 0 <= l < len(label_list)]
        for label, prediction in zip(labels, pred_logits)
    ]

    if len(predictions)!= len(true_labels):
      print(len(predictions), len(true_labels))
      print("not equal len!")

    # Calculate metrics
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [21]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

args = TrainingArguments(
	"test-ner",
	evaluation_strategy = "epoch",
	learning_rate=2e-5,
	per_device_train_batch_size=16,
	per_device_eval_batch_size=16,
	num_train_epochs=3,
	weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

# I changed this part to the evaluate function
metric = evaluate.load("seqeval")

trainer = Trainer(
    model,
	args,
	train_dataset=tokenized_datasets["train"],
	eval_dataset=tokenized_datasets["validation"],
	data_collator=data_collator,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer.train()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

c:\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.1657100915908813, 'eval_precision': 0.6666666666666666, 'eval_recall': 0.21739130434782608, 'eval_f1': 0.32786885245901637, 'eval_accuracy': 0.5866666666666667, 'eval_runtime': 0.4829, 'eval_samples_per_second': 20.707, 'eval_steps_per_second': 2.071, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.9382526278495789, 'eval_precision': 0.6666666666666666, 'eval_recall': 0.2608695652173913, 'eval_f1': 0.37500000000000006, 'eval_accuracy': 0.6333333333333333, 'eval_runtime': 0.6425, 'eval_samples_per_second': 15.564, 'eval_steps_per_second': 1.556, 'epoch': 2.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.8623010516166687, 'eval_precision': 0.8260869565217391, 'eval_recall': 0.41304347826086957, 'eval_f1': 0.5507246376811594, 'eval_accuracy': 0.7533333333333333, 'eval_runtime': 0.4369, 'eval_samples_per_second': 22.887, 'eval_steps_per_second': 2.289, 'epoch': 3.0}
{'train_runtime': 50.6538, 'train_samples_per_second': 4.146, 'train_steps_per_second': 0.296, 'train_loss': 1.2198683420817058, 'epoch': 3.0}


TrainOutput(global_step=15, training_loss=1.2198683420817058, metrics={'train_runtime': 50.6538, 'train_samples_per_second': 4.146, 'train_steps_per_second': 0.296, 'total_flos': 2232350935452.0, 'train_loss': 1.2198683420817058, 'epoch': 3.0})

# Save model

In [23]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [24]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id

# Model Prediction

In [25]:
json.dump(config, open("ner_model/config.json","w"))

In [26]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [27]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "The time taken for the service is very fast, easy-to-use, the app is within our reach"

ner_results = nlp(example)

for i in ner_results:
	print(i)

{'entity': 'I-CON', 'score': 0.35947707, 'index': 12, 'word': '-', 'start': 49, 'end': 50}
{'entity': 'I-CON', 'score': 0.37451065, 'index': 14, 'word': '-', 'start': 52, 'end': 53}
{'entity': 'I-CON', 'score': 0.36772236, 'index': 17, 'word': 'the', 'start': 58, 'end': 61}
